In [1]:
import pandas as pd

In [2]:

df = pd.read_csv('DATA/SMSSpamCollection', sep='\t',
                           names=["label", "message"])


FileNotFoundError: [Errno 2] No such file or directory: 'DATA/SMSSpamCollection'

In [32]:
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
df.shape

(5572, 2)

In [7]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm_notebook as tqdm

In [10]:
ps = PorterStemmer()

In [11]:



corpus = []  

with tqdm(total=len(df)) as pbar:
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        pbar.update(1)

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
corpus[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free']

### Using BOW MODEL

In [14]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [16]:
y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [17]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [18]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)


In [19]:
y_pred=spam_detect_model.predict(X_test)


In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [21]:
cm = confusion_matrix(y_test,y_pred)

In [22]:
cm

array([[946,   9],
       [  7, 153]], dtype=int64)

In [23]:
acc = accuracy_score(y_test, y_pred)

In [24]:
acc

0.9856502242152466

###  USING TF-IDF LIBRARY 

In [25]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X_tf = cv.fit_transform(corpus).toarray()


In [26]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size = 0.20, random_state = 0)


In [27]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)


In [28]:
y_pred=spam_detect_model.predict(X_test)


In [29]:
cm = confusion_matrix(y_test,y_pred)

In [30]:
cm

array([[955,   0],
       [ 34, 126]], dtype=int64)

In [31]:
acc = accuracy_score(y_test, y_pred)
acc

0.9695067264573991